## Puzzle 2: cube conundrum

### The puzzle, part 1

You're launched high into the atmosphere! The apex of your trajectory just barely reaches the surface of a large island floating in the sky. You gently land in a fluffy pile of leaves. It's quite cold, but you don't see much snow. An Elf runs over to greet you.

The Elf explains that you've arrived at Snow Island and apologizes for the lack of snow. He'll be happy to explain the situation, but it's a bit of a walk, so you have some time. They don't get many visitors up here; would you like to play a game in the meantime?

As you walk, the Elf shows you a small bag and some cubes which are either red, green, or blue. Each time you play this game, he will hide a secret number of cubes of each color in the bag, and your goal is to figure out information about the number of cubes.

To get information, once a bag has been loaded with cubes, the Elf will reach into the bag, grab a handful of random cubes, show them to you, and then put them back in the bag. He'll do this a few times per game.

You play several games and record the information from each game (your puzzle input). Each game is listed with its ID number (like the 11 in Game 11: ...) followed by a semicolon-separated list of subsets of cubes that were revealed from the bag (like 3 red, 5 green, 4 blue).

For example, the record of a few games might look like this:

Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
In game 1, three sets of cubes are revealed from the bag (and then put back again). The first set is 3 blue cubes and 4 red cubes; the second set is 1 red cube, 2 green cubes, and 6 blue cubes; the third set is only 2 green cubes.

The Elf would first like to know which games would have been possible if the bag contained only 12 red cubes, 13 green cubes, and 14 blue cubes?

In the example above, games 1, 2, and 5 would have been possible if the bag had been loaded with that configuration. However, game 3 would have been impossible because at one point the Elf showed you 20 red cubes at once; similarly, game 4 would also have been impossible because the Elf showed you 15 blue cubes at once. If you add up the IDs of the games that would have been possible, you get 8.

Determine which games would have been possible if the bag had been loaded with only 12 red cubes, 13 green cubes, and 14 blue cubes. What is the sum of the IDs of those games?



### The solution

First, write some code to parse the input into lines, games, and draws.  The store the result of each draw in a numpy structured array with integer columns 'blue', green' and 'red'.  To make the solution of most use to me, use the same structure as in MEGA to have games point to sets of draws within one large array.

Once we have that structure in place, it is trivial to select draws that are valid, and hence games that are valid.  Use a fourth boolean column in the draw array 'valid' to record whether it is a valid draw.

In [ ]:
import numpy as np

# Could count the lines ourselves, but as the number is given, this is a nice check.
n_line=100

infile='Puzzle2.txt'
colours=['blue','green','red']
max_cube={'blue':14,'green':13,'red':12}
draw_dtype=np.dtype([('blue',int),('green',int),('red',int),('valid',bool)])
draw_template=np.zeros(1,dtype=draw_dtype)

# Read in data (lines from input file)
with open(infile,'r') as f:
    lines=f.readlines()
assert len(lines)==n_line

# These arrays show which draws belong to which games, and whether a game is valid
game_first_draw=np.full(n_line,-1,int)
game_n_draw=np.full(n_line,-1,int)
game_valid=np.full(n_line,True,bool)

# Populate the above arrays.
# To do this, count the number of draws on each line which should equal the number of semi-colons + 1.
n_draw=0
n_game=0
for line in lines:
    game_first_draw[n_game]=n_draw
    game_n_draw[n_game]=line.count(';')+1
    n_draw+=game_n_draw[n_game]
    n_game+=1
assert n_game==n_line
print('n_game =',n_game)
print('n_draw =',n_draw)

# Now create the draws array
draws=np.full(n_draw,draw_template,dtype=draw_dtype)

# Run through the lines, splitting each into draws and populating the draws array
i_game=0
i_draw=0
for line in lines:
    len_prefix=line.find(':')
    line=line[len_prefix+1:]
    draws_in_line=line.split(';')
    for draw in draws_in_line:
        entries = draw.split(',')
        for entry in entries:
            for colour in colours:
                if colour in entry:
                    draws[colour][i_draw]=int(entry.strip('bluegrnd\n'))
        i_draw+=1
    i_game+=1
assert i_game==n_game
assert i_draw==n_draw

# Now set the valid marker
for draw in draws:
    if draw['blue']<=max_cube['blue'] and draw['green']<=max_cube['green'] and draw['red']<=max_cube['red']:
        draw['valid']=True
# A game is valid only if all its draws are
answer=0
for i_game in range(n_game):
    for draw in draws[game_first_draw[i_game]:game_first_draw[i_game]+game_n_draw[i_game]]:
        if draw['valid']==False: game_valid[i_game]=False
    if game_valid[i_game]==True:
        answer+=1+i_game
    else:
        #print(lines[i_game])
        pass
print('answer =',answer)

### The puzzle, part 2

The Elf says they've stopped producing snow because they aren't getting any water! He isn't sure why the water stopped; however, he can show you how to get to the water source to check it out for yourself. It's just up ahead!

As you continue your walk, the Elf poses a second question: in each game you played, what is the fewest number of cubes of each color that could have been in the bag to make the game possible?

Again consider the example games from earlier:

Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
In game 1, the game could have been played with as few as 4 red, 2 green, and 6 blue cubes. If any color had even one fewer cube, the game would have been impossible.
Game 2 could have been played with a minimum of 1 red, 3 green, and 4 blue cubes.
Game 3 must have been played with at least 20 red, 13 green, and 6 blue cubes.
Game 4 required at least 14 red, 3 green, and 15 blue cubes.
Game 5 needed no fewer than 6 red, 3 green, and 2 blue cubes in the bag.
The power of a set of cubes is equal to the numbers of red, green, and blue cubes multiplied together. The power of the minimum set of cubes in game 1 is 48. In games 2-5 it was 12, 1560, 630, and 36, respectively. Adding up these five powers produces the sum 2286.

For each game, find the minimum set of cubes that must have been present. What is the sum of the power of these sets?

### The solution, part 2

Now that we have the draws, this should be easy.  We just need the maximum of each colour over all draws for each game.

In [ ]:
game_power=np.zeros(n_game,int)
for i_game in range(n_game):
    max_blue=0
    max_green=0
    max_red=0
    for draw in draws[game_first_draw[i_game]:game_first_draw[i_game]+game_n_draw[i_game]]:
        max_blue=max(max_blue,draw['blue'])
        max_green=max(max_green,draw['green'])
        max_red=max(max_red,draw['red'])
    game_power[i_game]=max_blue*max_green*max_red
print('answer=',np.sum(game_power))

In [ ]:
game_power